In [1]:
import numpy as np
import cupy as cp
import mpmath as mp
import numba
from numba import vectorize, guvectorize

ModuleNotFoundError: No module named 'numba'

In [20]:
# NUMPY Vectorization with mpmath function
@np.vectorize
def my_ellipf(phi,m):
    y = mp.ellipf(phi,m)
    y = np.float(y)
    return y

def ellip_arg2(x):
    return -4*(1+x)/x**2

def simple_psi_x(z, x, beta):
    return  my_ellipf(alpha(z,x,beta),-4*(1+x)/x**2)

In [3]:
# Skip this cell if CSR2D/ is already added under PYHTONPATH 
import os, sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)  

In [4]:
from csr2d.core import psi_s, alpha, kappa
from csr2d.dist import lambda_p_Gauss

# Test with numpy arrays

In [22]:
gamma = 500
beta = (1-1/gamma**2)**(1/2)
rho = 1
sigmax = 10E-6
sigmaz = 10E-6

# Adjust the step size here
# Small steps might greatly increase computation time...
#dz = 0.1*sigmaz
#dx = 0.1*sigmax

N = 3
dz = 1.0E-4/N
dx = 1.0E-4/N

zvec = np.arange(-5*sigmaz, 5*sigmaz, dz)
xvec = np.arange(-5*sigmax, 5*sigmax, dx)
zm, xm = np.meshgrid(zvec, xvec, sparse=False, indexing='ij')

In [23]:
alpha(zm,xm,beta)

array([[-1.87502148e-05, -2.43056804e-05, -2.43054835e-05],
       [ 1.04164159e-05, -6.25003255e-06, -6.24999349e-06],
       [ 4.52571913e-02,  4.59688435e-02,  4.66865214e-02]])

In [24]:
ellip_arg2(xm)

array([[-1.599920e+09, -1.439976e+10, -1.440024e+10],
       [-1.599920e+09, -1.439976e+10, -1.440024e+10],
       [-1.599920e+09, -1.439976e+10, -1.440024e+10]])

In [25]:
simple_psi_x(zm,xm,beta)

array([[-1.73289096e-05, -1.49314281e-05, -1.49312468e-05],
       [ 1.01364093e-05, -5.77625902e-06, -5.77621483e-06],
       [ 2.04868472e-04,  7.75737345e-05,  7.77017227e-05]])

# Test with Cupy arrays

In [16]:
zvec2 = cp.arange(-5*sigmaz, 5*sigmaz, dz)
xvec2 = cp.arange(-5*sigmax, 5*sigmax, dx)
zm2, xm2 = cp.meshgrid(zvec2, xvec2, sparse=False, indexing='ij')

In [17]:
alpha(zm2,xm2,beta)

array([[-1.87502148e-05, -2.43056804e-05, -2.43054835e-05],
       [ 1.04164159e-05, -6.25003255e-06, -6.24999349e-06],
       [ 4.52571913e-02,  4.59688435e-02,  4.66865214e-02]])

In [18]:
ellip_arg2(xm2)

array([[-1.599920e+09, -1.439976e+10, -1.440024e+10],
       [-1.599920e+09, -1.439976e+10, -1.440024e+10],
       [-1.599920e+09, -1.439976e+10, -1.440024e+10]])

In [19]:
# This doesn't work
# Numpy vectorization doesn't work Cupy arrays...  "cp.vectorize" doesn't exist
simple_psi_x(zm2,xm2,beta)

ValueError: object __array__ method not producing an array

# Trying with numba vectorize 

### https://carpentries-incubator.github.io/gpu-speedups/01_CuPy_and_Numba_on_the_GPU/index.html
### https://numba.pydata.org/numba-doc/latest/user/vectorize.html

In [30]:
@numba.vectorize(['int64(int64, int64)',
                  'float32(float32, float32)',
                  'float64(float64, float64)'],
                  target='cuda')
def add_ufunc(x, y):
    return x + y

In [31]:
# Works with numpy arrays
# When target is set to cuda, GPU is used according to documentation ( see links above )
# So even though the arrays are numpy arrays, calculation seems to be done by GPU (?)
a = np.array([1, 2, 3, 4])
b = np.array([10, 20, 30, 40])
add_ufunc(a, b)  

array([11, 22, 33, 44])

In [35]:
# Works with cupy arrays
a = cp.array([1, 2, 3, 4])
b = cp.array([10, 20, 30, 40])
y = add_ufunc(a, b)  
cp.asnumpy(y)

array([11., 22., 33., 44.])

In [36]:
# This defintion doesn't work though...
@numba.vectorize(['float32(float32, float32)',
                  'float64(float64, float64)'],
                  target='cuda')
def my_ellipf(phi,m):
    y = mp.ellipf(phi,m)
    #y = np.float(y)
    return y

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'ellipf' of type Module(<module 'mpmath' from '/sdf/home/w/wlou/miniconda3/envs/test/lib/python3.8/site-packages/mpmath/__init__.py'>)

File "<ipython-input-36-80fb128aaee0>", line 6:
def my_ellipf(phi,m):
    y = mp.ellipf(phi,m)
    ^

During: typing of get attribute at <ipython-input-36-80fb128aaee0> (6)

File "<ipython-input-36-80fb128aaee0>", line 6:
def my_ellipf(phi,m):
    y = mp.ellipf(phi,m)
    ^


In [37]:
# This also doesn't work
@numba.vectorize(['float64(float64, float64)'],
                  target='cuda')
def my_ellipf(phi,m):
    yy = np.array([0.0, 0.0, 30.0, 40.0])
    return yy

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Use of unsupported NumPy function 'numpy.array' or unsupported use of the function.

File "<ipython-input-37-12e2517eef1f>", line 5:
def my_ellipf(phi,m):
    yy = np.array([0.0, 0.0, 30.0, 40.0])
    ^

During: typing of get attribute at <ipython-input-37-12e2517eef1f> (5)

File "<ipython-input-37-12e2517eef1f>", line 5:
def my_ellipf(phi,m):
    yy = np.array([0.0, 0.0, 30.0, 40.0])
    ^


### Possible solution ( seems like numba is quite reestrictive... )
### https://stackoverflow.com/questions/53876205/whats-wrong-in-this-code-unknown-attribute-array-of-type-modulemodule-num

# Also trying  guvectorize from numba

In [38]:
@guvectorize([('int64, int64, int64')], '(),()->()')
def g(x, y, out):
    out = x + y

In [40]:
# Why is the answer not 7...?
g(3,4) 

3